## Ejercicio 1: Función VaR y ES

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from IPython.display import display
from scipy.optimize import minimize

pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
def var_stocks(data, n_stocks, conf, long):
    rt = data.pct_change().dropna()
    stock_value = n_stocks * data.iloc[-1]
    portfolio_value = stock_value.sum()
    w = stock_value / portfolio_value
    portfolio_return = np.dot(w, rt.T)

    if long == 1:

        var_pct = np.percentile(portfolio_return, 100-conf)
        cvar_pct = np.abs(portfolio_return[portfolio_return < var_pct].mean())

        var_cash = portfolio_value * np.abs(var_pct)
        cvar_cash = portfolio_value * cvar_pct

    else:
        
        var_pct = np.percentile(portfolio_return, conf)
        cvar_pct = portfolio_return[portfolio_return > var_pct].mean()

        var_cash = portfolio_value * var_pct
        cvar_cash = portfolio_value * cvar_pct

    var_stocks_df = pd.DataFrame({
        "Métrica": ["VaR", "cVaR"],
        "Porcentaje": [np.abs(var_pct), cvar_pct],
        "cash": [var_cash, cvar_cash]
    })

    return var_stocks_df

In [3]:
stocks = ["AAPL", "TSLA", "AMD", "LMT", "JPM"]
data=yf.download(stocks, start="2020-01-01", end="2023-01-01")['Adj Close'][stocks]

n_stocks =[2193, 1211, 3221, 761, 1231]
conf = 95
long = True

[*********************100%***********************]  5 of 5 completed


In [4]:
var_stocks(data, n_stocks, conf, long)

,Métrica,Porcentaje,cash
0,VaR,0.03,"33,073.78"
1,cVaR,0.05,"53,131.45"


## Ejercicio 2: Rebalanceo de portafolios

In [5]:
rt = data.pct_change().dropna()
stock_value = n_stocks * data.iloc[-1]
portfolio_value = stock_value.sum()
w_original = stock_value / portfolio_value
target = 30000

In [6]:
def opt_target_var(weights):

    def get_var(weights):
        return np.abs(np.percentile(np.dot(weights, rt.T), 100-conf)) * portfolio_value if long else np.percentile(np.dot(weights, rt.T), conf) * portfolio_value

    # Restricciones
    constraints = [
        {"type": "eq", "fun": lambda w: np.sum(w) - 1},
        {"type": "eq", "fun": lambda w: get_var(w) - target}
    ]

    bounds = tuple((0, 1) for _ in range(len(n_stocks)))
    initial_guess = np.ones(len(n_stocks)) / len(n_stocks)

    # Optimización
    result = minimize(
        fun=get_var,
        x0=initial_guess,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        tol=1e-8
    )

    opt_weights = result.x

    return opt_weights, get_var(opt_weights)

In [7]:
opt_weights, opt_var = opt_target_var(w_original)

In [8]:
def rebalanceo(w_original, opt_weights):
    w_df = pd.DataFrame({
    "Peso Original": w_original,
    "Peso Óptimo": opt_weights,
    "Acciones (C/V)" : np.floor((opt_weights-w_original) * portfolio_value / data.iloc[-1])
    })
    return print(f'VaR con pesos óptimos: ${opt_var:.2f}'), display(w_df.T)

In [9]:
rebalanceo(w_original, opt_weights)

VaR con pesos óptimos: $30000.00


Ticker,AAPL,TSLA,AMD,LMT,JPM
Peso Original,0.25,0.13,0.18,0.31,0.14
Peso Óptimo,0.33,0.15,0.06,0.46,0.00
Acciones (C/V),762.00,188.00,"-2,167.00",373.00,"-1,212.00"


(None, None)

Para rebalancear el portafolio para tener nuestro VaR objetivo de $30,000 se realiza lo siguiente:

- Comprar 763 acciones de APPL.
- Comprar 188 acciones de TSLA. 
- Vender 2167 acciones de AMD.
- Comprar 373 acciones de LMT.
- Vender 1212 acciones de JPM.

## Ejercicio 3: Investigación conceptual

**Inciso a)**

- **¿Cuál es el objetivo principal del rebalanceo?**

Mantener el porcentaje asignado a los activos en línea con tu estrategia de inversión inicial, ajustándolos para que puedan cumplir tus objetivos de inversión y que el riesgo se matenga dentro de un nivel que consideres adecuado. En general ayuda a gestionar el riesgo de portafolio y evita una exposición excesiva a activos volátiles.

- **¿Por qué es importante implementarlo de manera sistemática?**

Evita el factor de la emociones en la toma de decisiones de inversión, pues ya hay reglas establecidas para hacer el rebalanceo, permite un buen monitoreo del portafolio y ayuda a mantenerlo en línea con tu estrategia de inversión y objetivos.

**Inciso b)**

- **¿Qué decisiones forman parte de una estrategia de rebalanceo?**

Definir la frecuencia de rebalanceo del portafolio, determinar si es necesario algún umbral de desviación máxima respecto a la estrategia y el nivel de ajuste que se hará al portafolio, ya sea total o parcial. Esto dependiendo de tus objetivos de inversión, aversión al riesgo, el plazo de inversión que tengas pensado, el nivel de ahorro que desees lograr y la exposición deseada hacia ciertos tipos de activos.

- **¿Cuáles son los métodos principales para realizar rebalanceos de forma sistemática?**
    - **Basado en el calendario:** Determinar un plazo de tiempo específico para hacer el rebalanceo de forma periódica. (trimestral, semestral, anual, etc.)
    - **Basado en un umbral:** Rebalancear solo si cruza un nivel de desviación máximo permitido.
    - **Combinación de ambos:** Monitorear por peridos de tiempo fijo y rebalancear si se desvía más de lo permitido de la asignación de activos establecida.


**Inciso c)**

- Además del rendimiento, riesgo y objetivos, ¿qué otro factor puede tener un gran impacto en el desempeño de un portafolio al
rebalancearlo?

Los costos de transacción pueden tener un efecto dañino en el rebalanceo, sobre todo en momentos de volatilidad alta, además de impuestos que se deban de pagar sobre ganancias de capital al momento de rebalanceo.

**Inciso d)**

- ¿Qué recomendaciones propone Vanguard para reducir los costos fiscales en un portafolio?

Rebalancear de forma menos frecuente, por ejemplo anual, además de hacer rebalanceos parciales enfocándose en ativos que estén muy por debajo o por encima de su objetivo.

- ¿Qué es el tax loss harvesting?

Busca aprovechar las pérdidas fiscales de tus activos, vendes activos depreciados para compensar las ganacias de capital de otros activos, lo cual reduce la carga fiscal. Por ejemplo, si perdiste $\$1000$ en una acción, las vendes para realizar la pérdida que compensa una ganancia de $\$1000$ sobre otro activo, dejando una ganancia neta de $\$0$ lo cual no pagaría impuestos.